In [62]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

In [63]:
def isNaN(num):
    return num != num

In [64]:
# read in data
pd_loan = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)
pd_loan.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0


In [ ]:
# cleaning Data
# first viewing categories of data which are object
categories = pd_loan.select_dtypes(include=['object'])
for x in categories:
    column = categories[x]
    print(x)
    print(column.nunique())
    

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [ ]:
# check the columns with NaN values
nrow = pd_loan_cln.shape[0]
for ptr_row in range(nrow):
    for ptr_col in pd_loan_cln.columns:
        if isNaN(pd_loan_cln.loc[ptr_row, ptr_col]):
            print('row {} & col {}'.format(ptr_row, ptr_col))

In [ ]:
# for 'object' of many types, converting them to numeric
pd_loan_cln = pd_loan.copy(deep=True)
pd_loan_cln['id'] = pd.to_numeric(pd_loan_cln['id'], errors='coerce')
pd_loan_cln['int_rate'] = pd.to_numeric(pd_loan_cln['int_rate'].str.strip('%'), errors='coerce')
pd_loan_cln['purpose'] = pd_loan_cln['purpose'].str
pd_loan_cln['title'] = pd_loan_cln['title'].str


In [ ]:
# drop columns with many types & unimportant in model building
pd_loan_cln.drop(['emp_title', 'url', 'desc', 'zip_code', 'earliest_cr_line', 
                       'revol_util', 'sub_grade', 'addr_state'], 1, inplace=True)
# drop additional columns 
pd_loan_cln.drop(['member_id', 'funded_amnt_inv', 'issue_d', 'purpose', 'title', 'initial_list_status', 'last_pymnt_d', 
              'next_pymnt_d', 'last_credit_pull_d'], 1, inplace=True)
#pd_loan_cln.drop(['verification_status_joint_Not Verified', 'verification_status_joint_Source Verified', 
#             'verification_status_joint_Verified', 'verification_status_Verified'], 1, inplace=True)
pd_loan_cln.drop(['tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
             'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75'], 1, inplace=True)
pd_loan_cln.drop(['collections_12_mths_ex_med', 'mths_since_last_major_derog', 'open_acc_6m', 'open_il_6m', 
                  'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 
                  'open_rv_24m', 'max_bal_bc', 'inq_last_12m', 'acc_open_past_24mths', 'mo_sin_old_il_acct',
                   'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'inq_last_6mths', 
                  'mths_since_last_delinq', 'mths_since_last_record'], 1, inplace=True)
pd_loan_cln.drop(['mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 
        'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'all_util', 'total_rev_hi_lim', 'inq_fi', 
        'total_cu_tl', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'collection_recovery_fee',
        'verification_status_joint', 'annual_inc_joint', 'dti_joint', 'emp_length'], 1, inplace=True)

In [ ]:
# removing last two rows
pd_loan_cln = pd_loan_cln[:-2]

In [ ]:
pd_loan_cln.columns

In [ ]:
pd.get_dummies(pd_loan_cln)

In [ ]:
# ready to build model
rfc = ensemble.RandomForestClassifier()
X = pd_loan_cln.drop('loan_status', 1)
Y = pd_loan_cln['loan_status']
X = pd.get_dummies(X)
X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10, error_score=np.nan)